In [40]:
import re
from datetime import datetime, timedelta
from functools import reduce
from typing import List

class RegexFail(Exception):
    pass

class InvalidFileType(Exception):
    pass

In [7]:
def get_leading_zeroes(month: int) -> str:
    if len(str(month)) > 1:
        return(month)
    else:  
        return(f"0{month}")

In [9]:
month = get_leading_zeroes(datetime.now().month)
yyyy = datetime.now().year
print(f"{yyyy}-{month}")

2022-02


In [17]:
#                     text / text / text / 4digits-2digits / text_4digits-2digits-2digits.(csv|parquet|zip)$ END WORD
#        bucket / table type / file type / year month      / table type _ year _ month _ day / file suffix END 
# https://regexr.com/
pattern = re.compile(r"\w+\/\w+\/\w+\/\d{4}-\d{2}\/(\w+)_\d{4}-\d{2}-\d{2}.(csv|parquet|json|zip)$")
string1 = "jacobsbucket97/sample_files/raw/2022-01/sample_files_2022-02-01.json"
string2 = "jacobsbucket97/sample_files/raw/2022-01/sample_files_2022-02-01.csv"
string3 = "jacobsbucket97/sample_files/raw/2022-01/sample_files_2022-02-01.parquet"
string4 = "jacobsbucket97/sample_files/raw/2022-01/sample_files_2022-02-01.gz"
string5 = "jacobsbucket97/sample_files/raw/2022-01/sample_files_2022-02-01.zip"
string6 = "jacobsbucket97/sample_files/raw/2022-01/sample_files_2022-02.parquet"

string_list = [string1, string2, string3, string4, string5]

for i in string_list:
    if re.match(pattern, i):
        print(f"{i} passed regex test")
    else:
        print(f"{i} failed regex test")

jacobsbucket97/sample_files/raw/2022-01/sample_files_2022-02-01.json passed regex test
jacobsbucket97/sample_files/raw/2022-01/sample_files_2022-02-01.csv passed regex test
jacobsbucket97/sample_files/raw/2022-01/sample_files_2022-02-01.parquet passed regex test
jacobsbucket97/sample_files/raw/2022-01/sample_files_2022-02-01.gz failed regex test
jacobsbucket97/sample_files/raw/2022-01/sample_files_2022-02-01.zip passed regex test


In [11]:
def check_filename(pattern: re.Pattern, filename: str):
    if not filename.endswith(("json", "csv", "parquet")):
        raise InvalidFileType(f"{filename} has an invalid file type: Please write as json, csv, or parquet")

    if re.search(pattern, filename):
        print(f"{filename} passed regex check")
        pass
    else:
        print(f"{filename} failed regex test")
        raise RegexFail(f"{filename} failed regex test")

In [26]:
check_filename(pattern, string6)

jacobsbucket97/sample_files/raw/2022-01/sample_files_2022-02.parquet failed regex test


RegexFail: jacobsbucket97/sample_files/raw/2022-01/sample_files_2022-02.parquet failed regex test

In [43]:
incorrectPart = None
pattern2 = "\w+\/\w+\/\w+\/\d{4}-\d{2}\/(\w+)_\d{4}-\d{2}-\d{2}.(csv|parquet|json|zip)$"
pattern2 = pattern2.replace('\.','|\.|').replace('_','|_|')
for i in re.split(pattern2, string4):
    if len(i)>1:
        incorrectPart = i
        print(incorrectPart)

sample_files_2022


TypeError: object of type 'NoneType' has no len()

In [37]:
pattern = re.compile(r"\w+\/\w+\/\w+\/\d{4}-\d{2}\/(\w+)_\d{4}-\d{2}-\d{2}.(csv|parquet|json|zip)$")
pattern = re.compile(r"(\w+)_\d{4}-\d{2}-\d{2}.(csv|parquet|json|zip)$")

pattern = [
        re.compile(r"\w+\/"),
        re.compile(r"\w+\/"),
        re.compile(r"\w+\/"),
        re.compile(r"\d{4}-\d{2}\/"),
        re.compile(r"(\w+)_\d{4}-\d{2}-\d{2}"),
        re.compile(r".(csv|parquet|json|zip)$")
        ]

# 'fn' is the file name matched so far
def reductor(fn, rx):
    if fn is None:
        return None
    mo = rx.match(fn)
    if mo is None:
        print(f"File name mismatch: got {fn}, expected {rx.pattern}")
        return None
    # proceed with the remainder of the string
    return fn[mo.end():]

In [87]:
def validate_file(filename: str, pattern: List[re.Pattern]) -> bool:
    def reductor(fn, rx):
        if fn is None:
            return None
        mo = rx.match(fn)
        if mo is None:
            print(f"File name mismatch: got {fn}, expected {rx.pattern}")
            raise RegexFail(f"File name mismatch: got {fn}, expected {rx.pattern}")
        # proceed with the remainder of the string
        return fn[mo.end():]

    if not filename.endswith(("json", "csv", "parquet")):
        raise InvalidFileType(f"{filename} has an invalid file type: expecting json, csv, or parquet")
    
    
    validate_obj = lambda fn: reduce(reductor, pattern, fn) is not None
    is_filename_correct = validate_obj(filename)

    return is_filename_correct

In [89]:
bb2 = validate_file(string4, pattern)

InvalidFileType: jacobsbucket97/sample_files/raw/2022-01/sample_files_2022-02-01.gz has an invalid file type: expecting json, csv, or parquet

In [54]:
validFile = lambda fn: reduce(reductor, pattern, fn) is not None
print(validFile("jacobsbucket97/sample_files/raw/2022-01/sample_files_2022-02.json"))

File name mismatch: got sample_files_2022-02.json, expected (\w+)_\d{4}-\d{2}-\d{2}
False


In [5]:
def verifyFileName(filename__, pattern__):
    '''
    Verifies if a file name is correct
    :param filename__: file name 
    :param pattern__: pattern
    :return: empty string if file name is correct, otherwise the incorrect part of file
    '''
    incorrectPart =""
    pattern = pattern__.replace('\.','|\.|').replace('_','|_|')
    for i in re.split(pattern, filename__):
        print(i)
        if len(i)>1:
            incorrectPart = i
    return incorrectPart

pattern3 = "\w+\/\w+\/\w+\/\d{4}-\d{2}\/(\w+)_\d{4}-\d{2}-\d{2}.(csv|parquet|json|zip)$"
verifyFileName(string2, pattern3)


sample_files_2022
None


TypeError: object of type 'NoneType' has no len()

In [10]:
pattern3 = re.compile(r"\w+\/\w+\/\w+\/\d{4}-\d{2}\/(\w+)_\d{4}-\d{2}-\d{2}.(csv|parquet|json|zip)$")
pattern4 = re.split(pattern3, string1)

In [3]:
#                     text / text / (validated|invalidated) / 4digits-2digits / text_4digits-2digits-2digits.(csv|parquet|zip)$ END WORD
pattern = re.compile(r"\w+\/\w+\/(validated|invalidated)/\d{4}-\d{2}\/(\w+)_\d{4}-\d{2}-\d{2}.(csv|parquet|json|zip)$")
string1 = "jacobsbucket97/sample_files/validated/2022-01/sample_files_2022-02-01.json"
string2 = "jacobsbucket97/sample_files/invalidated/2022-01/sample_files_2022-02-01.csv"
string3 = "jacobsbucket97/sample_files/validated/2022-01/sample_files_2022-02-01.parquet"
string4 = "jacobsbucket97/sample_files/invalidated/2022-01/sample_files_2022-02-01.gz"
string5 = "jacobsbucket97/sample_files/blahblahblah/2022-01/sample_files_2022-02-01.zip"

string_list = [string1, string2, string3, string4, string5]

for i in string_list:
    if re.match(pattern, i):
        print(f"{i} passed regex test")
    else:
        print(f"{i} failed regex test")

jacobsbucket97/sample_files/validated/2022-01/sample_files_2022-02-01.json passed regex test
jacobsbucket97/sample_files/invalidated/2022-01/sample_files_2022-02-01.csv passed regex test
jacobsbucket97/sample_files/validated/2022-01/sample_files_2022-02-01.parquet passed regex test
jacobsbucket97/sample_files/invalidated/2022-01/sample_files_2022-02-01.gz failed regex test
jacobsbucket97/sample_files/blahblahblah/2022-01/sample_files_2022-02-01.zip failed regex test


In [26]:
# find how many times a pattern shows up in a string
pattern1 = 'baby'
pattern2 = '.csv'
pattern3 = '.csv$'
pattern4 = '\w+'
string1 = "hi .csv this baby is a baby .csv file for .csv"


print(len(re.findall(pattern, string1)))  # 2 baby
print(len(re.findall(pattern2, string1))) # 3 csv
print(len(re.findall(pattern3, string1))) # 1 csv that ends the string
print(len(re.findall(pattern4, string1))) # 11 unique string combinations

2
3
1
4


In [ ]:
import re

dict1 = {"key1": "value1", "key2": "value2", "key3": "value3"}

for k, v in dict1.items():
    if k.startswith('key'):
        loop_key = re.split(r'(\d+)', k) # split key1 into ['key', '1']
        print(loop_key)
        print(f"{loop_key[0]} starts with key")
        print(f"{loop_key[1]} is the first number")

In [53]:
import re
str1 = "testing 123 please remove all urls https://developer.twitter.com/ testing 123. https://google.com hahah"
str2 = "searches for IPhone 5s search results"

str1_method1 = ' '.join(x for x in str1.split() if not x.startswith('https'))
str1_method2 = re.sub("\bhttps\w+", "", "https://google.com haha")